In [1]:
import pandas as pd

# Read data from files 
train = pd.read_csv( "labeledTrainData.tsv", header=0, 
 delimiter="\t", quoting=3 )
test = pd.read_csv( "testData.tsv", header=0, delimiter="\t", quoting=3 )
unlabeled_train = pd.read_csv( "unlabeledTrainData.tsv", header=0, 
 delimiter="\t", quoting=3 )

# Verify the number of reviews that were read (100,000 in total)
print "Read %d labeled train reviews, %d labeled test reviews, " \
"and %d unlabeled reviews\n" % (train["review"].size,  
 test["review"].size, unlabeled_train["review"].size )

Read 25000 labeled train reviews, 25000 labeled test reviews, and 50000 unlabeled reviews



In [2]:
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

def review_to_wordlist( review, remove_stopwords=False ):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(review,'lxml').get_text().encode('ascii', 'ignore').decode('ascii')
    #  
    # 2. Remove non-letters
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    #
    # 3. Convert words to lower case and split them
    words = review_text.lower().split()
    #
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    #
    # 5. Return a list of words
    return(words)

In [3]:
import nltk.data
# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

In [4]:
import string
from gensim.models.phrases import Phrases
from gensim.models import Word2Vec
from nltk.corpus import stopwords
bigram = Phrases()
# Define a function to split a review into parsed sentences
def review_to_sentences( bigram,review, tokenizer, remove_stopwords=False ):
    # Function to split a review into parsed sentences. Returns a 
    # list of sentences, where each sentence is a list of words
    #
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    review=re.sub(r'[^\x00-\x7f]',r' ',review)
    raw_sentences = tokenizer.tokenize(review.strip())
    #
    # 2. Loop over each sentence
    sentences = []
    
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentence=review_to_wordlist( raw_sentence,remove_stopwords )
            sentences.append(sentence)
            bigram.add_vocab([sentence])
    #
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences

C:\ProgramData\Anaconda2\lib\site-packages\gensim\utils.py:855: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [5]:
sentences = []  # Initialize an empty list of sentences

print "Parsing sentences from training set"
for review in train["review"]:
    sentences += review_to_sentences(bigram,review, tokenizer)

print "Parsing sentences from unlabeled set"
for review in unlabeled_train["review"]:
    sentences += review_to_sentences(bigram,review, tokenizer)

Parsing sentences from training set


C:\ProgramData\Anaconda2\lib\site-packages\bs4\__init__.py:219: UserWarning: "." looks like a filename, not markup. You shouldprobably open this file and pass the filehandle intoBeautiful Soup.
  'Beautiful Soup.' % markup)
C:\ProgramData\Anaconda2\lib\site-packages\bs4\__init__.py:219: UserWarning: "..." looks like a filename, not markup. You shouldprobably open this file and pass the filehandle intoBeautiful Soup.
  'Beautiful Soup.' % markup)
C:\ProgramData\Anaconda2\lib\site-packages\bs4\__init__.py:282: UserWarning: "http://www.happierabroad.com"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


Parsing sentences from unlabeled set


C:\ProgramData\Anaconda2\lib\site-packages\bs4\__init__.py:282: UserWarning: "http://www.archive.org/details/LovefromaStranger"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
C:\ProgramData\Anaconda2\lib\site-packages\bs4\__init__.py:282: UserWarning: "http://www.loosechangeguide.com/LooseChangeGuide.html"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
C:\ProgramData\Anaconda2\lib\site-packages\bs4\__init__.py:219: UserWarning: "... ..." looks like a filename, not markup. You shouldprobably open this file and pass the filehandle intoBeautiful Soup.
  'Beautiful Soup.' % markup)
C:\ProgramData\Anaconda

In [12]:
len(sentences)

795317

#### Word vector Training using phrases

In [13]:
# Import the built-in logging module and configure it so that Word2Vec 
# creates nice output messages
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

# Initialize and train the model (this will take some time)
from gensim.models import word2vec
print "Training model..."
model = word2vec.Word2Vec(bigram[sentences], workers=num_workers,size=num_features, min_count = min_word_count,window = context, sample = downsampling)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "300features_40minwords_10context_phrases"
model.save(model_name)

2017-03-22 18:05:53,943 : INFO : collecting all words and their counts
2017-03-22 18:05:53,953 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types


Training model...


2017-03-22 18:05:55,714 : INFO : PROGRESS: at sentence #10000, processed 212804 words, keeping 21630 word types
2017-03-22 18:05:57,471 : INFO : PROGRESS: at sentence #20000, processed 426299 words, keeping 31596 word types
2017-03-22 18:05:59,095 : INFO : PROGRESS: at sentence #30000, processed 633297 words, keeping 38610 word types
2017-03-22 18:06:00,776 : INFO : PROGRESS: at sentence #40000, processed 847213 words, keeping 44647 word types
2017-03-22 18:06:02,405 : INFO : PROGRESS: at sentence #50000, processed 1053991 words, keeping 49327 word types
2017-03-22 18:06:04,053 : INFO : PROGRESS: at sentence #60000, processed 1263018 words, keeping 53427 word types
2017-03-22 18:06:05,707 : INFO : PROGRESS: at sentence #70000, processed 1473470 words, keeping 56968 word types
2017-03-22 18:06:07,328 : INFO : PROGRESS: at sentence #80000, processed 1680436 words, keeping 60179 word types
2017-03-22 18:06:09,071 : INFO : PROGRESS: at sentence #90000, processed 1891232 words, keeping 6334

#### Word vectors with phrases

In [14]:
from gensim.models import Word2Vec
model = Word2Vec.load("300features_40minwords_10context_phrases")

2017-03-22 18:09:23,701 : INFO : loading Word2Vec object from 300features_40minwords_10context_phrases
2017-03-22 18:09:23,880 : INFO : loading wv recursively from 300features_40minwords_10context_phrases.wv.* with mmap=None
2017-03-22 18:09:23,881 : INFO : setting ignored attribute syn0norm to None
2017-03-22 18:09:23,884 : INFO : setting ignored attribute cum_table to None
2017-03-22 18:09:23,888 : INFO : loaded 300features_40minwords_10context_phrases


In [15]:
import numpy as np  # Make sure that numpy is imported

def makeFeatureVec(words, model, num_features):
    # Function to average all of the word vectors in a given
    # paragraph
    #
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,),dtype="float32")
    #
    nwords = 0.
    # 
    # Index2word is a list that contains the names of the words in 
    # the model's vocabulary. Convert it to a set, for speed 
    index2word_set = set(model.wv.vocab.keys())
    #
    # Loop over each word in the review and, if it is in the model's
    # vocaublary, add its feature vector to the total
    for word in words:
        if word in index2word_set: 
            nwords = nwords + 1.
            featureVec = np.add(featureVec,model.wv[word])
    # 
    # Divide the result by the number of words to get the average
    featureVec = np.divide(featureVec,nwords)
    return featureVec


def getAvgFeatureVecs(reviews, model, num_features):
    # Given a set of reviews (each one a list of words), calculate 
    # the average feature vector for each one and return a 2D numpy array 
    # 
    # Initialize a counter
    counter = 0.
    # 
    # Preallocate a 2D numpy array, for speed
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    # 
    # Loop through the reviews
    for review in reviews:
       #
       # Print a status message every 1000th review
       if counter%1000. == 0. :
           print "Review %d of %d" % (counter, len(reviews))
       # 
       # Call the function (defined above) that makes average feature vectors
       reviewFeatureVecs[counter] = makeFeatureVec(review, model, \
           num_features)
       #
       # Increment the counter
       counter = counter + 1.
    return reviewFeatureVecs

In [16]:
# ****************************************************************
# Calculate average feature vectors for training and testing sets,
# using the functions we defined above. Notice that we now use stop word
# removal.

clean_train_reviews = []
for review in train["review"]:
    clean_train_reviews.append( review_to_wordlist( review, \
        remove_stopwords=True ))

trainDataVecs = getAvgFeatureVecs( clean_train_reviews, model, num_features )

print "Creating average feature vecs for test reviews"
clean_test_reviews = []
for review in test["review"]:
    clean_test_reviews.append( review_to_wordlist( review, \
        remove_stopwords=True ))

testDataVecs = getAvgFeatureVecs( clean_test_reviews, model, num_features )

Review 0 of 25000
Review 1000 of 25000
Review 2000 of 25000
Review 3000 of 25000
Review 4000 of 25000
Review 5000 of 25000
Review 6000 of 25000
Review 7000 of 25000
Review 8000 of 25000
Review 9000 of 25000
Review 10000 of 25000
Review 11000 of 25000
Review 12000 of 25000
Review 13000 of 25000
Review 14000 of 25000
Review 15000 of 25000
Review 16000 of 25000
Review 17000 of 25000
Review 18000 of 25000
Review 19000 of 25000
Review 20000 of 25000
Review 21000 of 25000
Review 22000 of 25000
Review 23000 of 25000
Review 24000 of 25000
Creating average feature vecs for test reviews
Review 0 of 25000
Review 1000 of 25000
Review 2000 of 25000
Review 3000 of 25000
Review 4000 of 25000
Review 5000 of 25000
Review 6000 of 25000
Review 7000 of 25000
Review 8000 of 25000
Review 9000 of 25000
Review 10000 of 25000
Review 11000 of 25000
Review 12000 of 25000
Review 13000 of 25000
Review 14000 of 25000
Review 15000 of 25000
Review 16000 of 25000
Review 17000 of 25000
Review 18000 of 25000
Review 1900

In [17]:
from sklearn.svm import LinearSVC
print "Fitting a SVM to labeled training data..."
#forest = forest.fit( trainDataVecs, train["sentiment"] )
SVM=LinearSVC(dual=False)
SVM.fit(trainDataVecs, train["sentiment"])
# Test & extract results 
result = SVM.predict( testDataVecs )

Fitting a SVM to labeled training data...


In [18]:
# Write the test results 
output = pd.DataFrame( data={"id":test["id"], "sentiment":result} )
output.to_csv( "Word2Vec_phrases_AverageVectors_SVM.csv", index=False, quoting=3 )